In [ ]:
from keys import db_user, db_password, db_name, db_host, db_port
import psycopg2
import time
import requests

In [ ]:
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()

In [ ]:
api = "https://api.pushshift.io/reddit/search/comment?author="
#4/1/2015 00:00:00
after_time = 1427860800
#1/1/2018 00:00:00
final_time = 1514782800
#response limit
response_limit = 15

In [ ]:
sql = """select norm_user_name from public.norm_users"""

In [ ]:
# Get all sus_user_names and save to list
names = []
cur.execute(sql)
for row in cur:
    names.append(row[0])

In [ ]:
sql_insert = """
insert into norm_user_comments (author, comment_body, comment_id, created_utc, link_id, subreddit_id, subreddit) 
VALUES(%s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
for name in names[:10000]:
    counter = after_time
    user_comment_count = 0
    response_length = 1
    try:
        while counter < final_time and user_comment_count < 15 and response_length > 0:
            request = requests.get(f'{api}{name}&after={counter}&before={final_time}&size={response_limit}')
            response = request.json()
            submissions = response['data']
            response_length = len(submissions)
            try:
                counter = submissions[-1]['created_utc']
                for submission in submissions:
                    try:
                        cur.execute(sql_insert, (submission['author'], submission['body'], submission['id'], submission['created_utc'], submission['link_id'], submission['subreddit_id'], submission['subreddit']))
                        conn.commit()
                        user_comment_count += 1
                    except Exception as e:
                        print(f'submission error ---->{e}')
                        next
            except Exception as e:
                print(f'user error ---->{e}')
                time.sleep(1)
                next
    except Exception as e:
        print(f'query error ---->{e}, {request.status_code}, retry after ')
        time.sleep(1)
        next
